In [1]:
%cd  /mnt/mnemo5/tao/MLG/task_1/utils


/mnt/mnemo5/tao/MLG/task_1/utils


#  use DeepHistone

In [2]:
import numpy as np
import torch
from torchsummary import summary
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import scipy
from scipy import stats
from tqdm import tqdm
import copy
import time
import logging

/mnt/mnemo5/tao/anaconda3/envs/py309_MLG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%reload_ext autoreload
%autoreload 2


from data_loader import *
from dataset import HistoneDataset_returngenenames
from histone_loader import*
from stratification import *


from modified_DeepHistone_model import DeepHistone
from DeepHistone_opt1 import DeepHistone_opt1
from modified_DeepHistone_utils import model_train,model_eval,model_predict
from modified_DeepHistone_utils import get_reshaped_data
from modified_DeepHistone_utils import get_dict_from_data
from modified_DeepHistone_utils import save_model

In [4]:

model_save_folder="../data/DeepHistone/"
prefix="basic-model-"#"opt1-model-" #"basic-model-"

In [5]:
time_stamp=time.strftime("%Y%m%d-%H%M%S")

Log_Format = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(level=logging.INFO, filename=f"{model_save_folder}{prefix}time{time_stamp}.log",
                    filemode="a",format=Log_Format) # cant save into file in jupyter notebook through 

In [6]:
valid_chr=[5,20]
test_chr=[2]
train_chr=[i for i in range(1,23) if (i not in valid_chr+test_chr)]
logging.info(f"train_chr:{train_chr}valid_chr:{valid_chr}test_chr:{test_chr}")

all_genes = load_train_genes()
all_genes.head(n=3)

,gene_name,chr,gene_start,gene_end,TSS_start,TSS_end,strand,gex,cell_line
0,SLC20A1,2,112645939,112663825,112658362,112658412,+,0.000000,1
1,C11orf58,11,16613132,16758340,16738643,16738693,+,2239.103328,1
2,ZSCAN9,6,28224886,28233487,28225263,28225313,+,19.798064,1


In [7]:
# Get genes
train_genes=filter_genes_by_chr(all_genes,train_chr)
valid_genes=filter_genes_by_chr(all_genes,valid_chr)
test_genes=filter_genes_by_chr(all_genes,test_chr)

train_genes=train_genes.iloc[0:5000,:] # for testing reason

n_genes_train, _ = np.shape(train_genes)
n_genes_valid, _ = np.shape(valid_genes)
n_genes_test, _ = np.shape(test_genes)
logging.info(f"train_genes.shape:{train_genes.shape}valid_genes.shape:{valid_genes.shape}test_genes.shape:{test_genes.shape}")



In [8]:
# # Get genes
# total_train_genes, test_genes = chromosome_splits(test_size=0.01)
# n_total_train_genes=total_train_genes.shape[0]
# train_genes = total_train_genes.iloc[0:int(0.8*n_total_train_genes),:]
# valid_genes = total_train_genes.iloc[int(0.8*n_total_train_genes):,:]

# n_genes_train, _ = np.shape(train_genes)
# n_genes_valid, _ = np.shape(valid_genes)
# n_genes_test, _ = np.shape(test_genes)
# print(train_genes.shape,valid_genes.shape,test_genes.shape)



In [9]:
left_flank_size = 1000#500#1000
right_flank_size = 1000#500#1000
seq_bin_size=left_flank_size+right_flank_size
histone_bin_size = 100 #100 ,20 ,5,1

seq_bins=seq_bin_size
assert seq_bin_size % histone_bin_size==0
histone_bins=int(seq_bin_size/histone_bin_size)
logging.info(f"seq_bins:{seq_bins}histone_bins:{histone_bins}")

In [10]:
%%time
# Load train data
train_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(train_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_train)

# Load valid data
valid_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(valid_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_valid)

# Load test data
test_dataloader = torch.utils.data.DataLoader(
    HistoneDataset_returngenenames(test_genes,left_flank_size=left_flank_size,right_flank_size=right_flank_size,bin_size=histone_bin_size,use_seq=True), 
    shuffle=False, batch_size=n_genes_valid)

CPU times: user 4.46 s, sys: 1.2 s, total: 5.66 s
Wall time: 5.66 s


In [11]:
%%time 
# Run train loader
x_train_histone,x_train_seq,y_train,train_index=get_reshaped_data(dataloader=train_dataloader)

# Run valid loader
x_valid_histone,x_valid_seq,y_valid,valid_index=get_reshaped_data(dataloader=valid_dataloader)

# Run test loader
x_test_histone,x_test_seq,y_test,test_index=get_reshaped_data(dataloader=test_dataloader)



CPU times: user 29.6 s, sys: 410 ms, total: 30 s
Wall time: 30 s


In [12]:
print(len(train_index),len(valid_index),len(test_index))


5000 2748 2406


In [13]:
dna_dict= get_dict_from_data(train_index,valid_index,test_index,
                             x_train_seq,x_valid_seq,x_test_seq)

histone_dict= get_dict_from_data(train_index,valid_index,test_index,
                             x_train_histone,x_valid_histone,x_test_histone)
gex_dict = get_dict_from_data(train_index,valid_index,test_index,
                             y_train,y_valid,y_test)

In [14]:
#print(dna_dict['1_FERMT2'].shape,histone_dict['1_FERMT2'].shape,gex_dict['1_FERMT2'].shape)

In [ ]:


use_gpu = torch.cuda.is_available()


batchsize=50#10000 # 20, 30
epochs=1 #50

logging.info('Begin training model...')
if prefix=="basic-model-":#"opt1-model-" "basic-model-"
	model = DeepHistone(use_gpu,bin_list=[seq_bins,histone_bins])
elif prefix=="opt1-model-":
	model = DeepHistone_opt1(use_gpu,bin_list=[seq_bins,histone_bins])
best_model = copy.deepcopy(model)
best_valid_spearmanr=0
best_valid_loss = float('Inf')



for epoch in tqdm(range(epochs)):
	np.random.shuffle(train_index)
	train_loss= model_train(train_index,model,batchsize,dna_dict,histone_dict,gex_dict,)
	valid_loss,valid_gex,valid_pred= model_eval(valid_index, model,batchsize,dna_dict,histone_dict,gex_dict,)
	valid_spearmanr= scipy.stats.spearmanr(valid_pred , valid_gex ).correlation
	logging.info(f"epoch:{epoch} valid_loss:{valid_loss} valid_spearmanr:{valid_spearmanr}")
	if valid_spearmanr >best_valid_spearmanr:
		best_model = copy.deepcopy(model)

	if valid_loss < best_valid_loss: 
		early_stop_time = 0
		best_valid_loss = valid_loss	
	else:
		model.updateLR(0.1)
		early_stop_time += 1
		if early_stop_time >= 5: break

	logging.info(f"early_stop_time:{early_stop_time}")
 

# DeepHistone(Dense,Dense) is used.
# self.bins 2000
# self.out_size:128000
# self.bins 20
# self.out_size:1280
# combined_len: 129280
#   0%|                                                    | 0/1 [00:00<?, ?it/s]
# batch_idx: 0
# self.SeqOrDnase: seq ;self.seq.size torch.Size([50, 1, 4, 2000])
# self.conv1.size torch.Size([50, 128, 1, 2000])
# self.block1.size torch.Size([50, 512, 1, 2000])
# self.trans1.size torch.Size([50, 256, 1, 500])
# seq.out.size: 50 256 1 500
# flat_seq.size: torch.Size([50, 128000])
# dns.size: 50 7 20
# self.SeqOrDnase: dnase ;self.seq.size torch.Size([50, 1, 7, 20])
# self.conv1.size torch.Size([50, 128, 1, 20])
# self.block1.size torch.Size([50, 512, 1, 20])
# self.trans1.size torch.Size([50, 256, 1, 5])
# dnase.out.size: 50 256 1 5
# flat_dns.size torch.Size([50, 1280])
# combined.size: torch.Size([50, 129280])


  0%|                                                    | 0/1 [00:00<?, ?it/s]

batch_idx: 0


In [51]:
('Begin predicting...')
test_gex,test_pred = model_predict(test_index,best_model,batchsize,dna_dict,histone_dict,gex_dict,)	
test_score = scipy.stats.spearmanr(test_pred , test_gex ).correlation

print('Spearman Correlation Score: {}'.format(test_score))

Spearman Correlation Score: 0.6174819305753333


In [ ]:
('Begin predicting...')
test_gex,test_pred = model_predict(test_index,model,batchsize,dna_dict,histone_dict,gex_dict,)	
test_score = scipy.stats.spearmanr(test_pred , test_gex ).correlation

print('Spearman Correlation Score: {}'.format(test_score))

Spearman Correlation Score: 0.7127891842974753


In [16]:
print(model.forward_fn.parameters)

<bound method Module.parameters of NetDeepHistone_opt1(
  (seq_map): ModuleDense_opt1(
    (conv1): Sequential(
      (0): Conv2d(1, 128, kernel_size=(4, 9), stride=(1, 1), padding=(0, 4))
    )
    (block1): DenseBlock(
      (layer): Sequential(
        (0): BasicBlock(
          (block): Sequential(
            (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
            (2): Conv2d(128, 128, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4))
          )
        )
        (1): BasicBlock(
          (block): Sequential(
            (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
            (2): Conv2d(256, 128, kernel_size=(1, 9), stride=(1, 1), padding=(0, 4))
          )
        )
        (2): BasicBlock(
          (block): Sequential(
            (0): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (1): ReLU()
    

In [69]:
summary(model.forward_fn.seq_map)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       4,736
├─DenseBlock: 1-2                        --
|    └─Sequential: 2-2                   --
|    |    └─BasicBlock: 3-1              147,840
|    |    └─BasicBlock: 3-2              295,552
|    |    └─BasicBlock: 3-3              443,264
├─Sequential: 1-3                        --
|    └─BatchNorm2d: 2-3                  1,024
|    └─ReLU: 2-4                         --
|    └─Conv2d: 2-5                       131,328
|    └─MaxPool2d: 2-6                    --
├─DenseBlock: 1-4                        --
|    └─Sequential: 2-7                   --
|    |    └─BasicBlock: 3-4              590,592
|    |    └─BasicBlock: 3-5              1,180,928
|    |    └─BasicBlock: 3-6              1,771,264
├─Sequential: 1-5                        --
|    └─BatchNorm2d: 2-8                  2,048
|    └─ReLU: 2-9                         --
|    └─Conv2d: 2-10    

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       4,736
├─DenseBlock: 1-2                        --
|    └─Sequential: 2-2                   --
|    |    └─BasicBlock: 3-1              147,840
|    |    └─BasicBlock: 3-2              295,552
|    |    └─BasicBlock: 3-3              443,264
├─Sequential: 1-3                        --
|    └─BatchNorm2d: 2-3                  1,024
|    └─ReLU: 2-4                         --
|    └─Conv2d: 2-5                       131,328
|    └─MaxPool2d: 2-6                    --
├─DenseBlock: 1-4                        --
|    └─Sequential: 2-7                   --
|    |    └─BasicBlock: 3-4              590,592
|    |    └─BasicBlock: 3-5              1,180,928
|    |    └─BasicBlock: 3-6              1,771,264
├─Sequential: 1-5                        --
|    └─BatchNorm2d: 2-8                  2,048
|    └─ReLU: 2-9                         --
|    └─Conv2d: 2-10    

In [ ]:

# print('Begin saving...')
# np.savetxt(f"{model_save_folder}label.txt", valid_gex, fmt='%d', delimiter='\t')
# np.savetxt(f"{model_save_folder}pred.txt", valid_pred, fmt='%.4f', delimiter='\t')
# save_model(model=best_model,epoch=epoch,seq_bins=seq_bins,histone_bins=histone_bins,
#             model_save_folder=model_save_folder,prefix="",suffix="best")
# save_model(model=best_model,epoch=epoch,seq_bins=seq_bins,histone_bins=histone_bins,
#             model_save_folder=model_save_folder,prefix="",suffix="final")

# print('Finished.')


Begin saving...
Finished.
